In [32]:
!pip install python-dotenv
!pip install openai
!pip install langchain
!pip install langchain-google-genai

In [7]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

False

In [16]:
from langchain.llms.openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

In [17]:
# Create a .env file and write the OpenAI API key into it
with open(".env", "w") as f:
    f.write("OPENAI_API_KEY=YOUR API KEY")


In [ ]:
# Retrieve the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Print to check if the key is loaded correctly
print(f"Loaded OpenAI API key: {openai_api_key}")


In [19]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Initialize OpenAI with the API key
llm_open_ai = OpenAI(openai_api_key=openai_api_key, max_tokens=1024)


In [23]:
llm = llm_open_ai

In [24]:
PROMPT_TEMPLATE_TEXT = """
Generate a table in CSV format from the following bank statement data.

Add a column called "Category" and populate it with one of the following values:
[Groceries, Transport, Entertainment, Shopping, Utilities, Eating Out, Unknown]

{statement}
"""

In [25]:
CODING_TEMPLATE_TEXT = """
First, hard-code this data as a Python variable called 'items', with the category name and value.
Then write a Python script to sum this data by 'Category' and print the results.

{categorized_transactions}
"""

In [26]:
# Create the individual prompt templates.
categorization_template = PromptTemplate.from_template(PROMPT_TEMPLATE_TEXT)
coding_template = PromptTemplate.from_template(CODING_TEMPLATE_TEXT)

In [27]:
# Create the chains.
categorization_chain = LLMChain(llm=llm, prompt=categorization_template)
coding_chain = LLMChain(llm=llm, prompt=coding_template)

<ipython-input-27-2cb1f05843aa>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  categorization_chain = LLMChain(llm=llm, prompt=categorization_template)


In [29]:
# Join them into a sequential chain.
overall_chain = SimpleSequentialChain(
    chains=[categorization_chain, coding_chain], verbose=True)

In [30]:
# Load the bank statement data.
with open("bank_statement.csv", "r") as f:
    bank_statement_data = f.read()

In [31]:
# Run the chain using the bank statement data as input.
overall_chain.run(bank_statement_data)

<ipython-input-31-2203e446f967>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain.run(bank_statement_data)




> Entering new SimpleSequentialChain chain...
Date,Description,Amount (USD),Category
2024-01-01,Walmart,35.20,Groceries
2024-01-02,Shell Gas Station,45.00,Transport
2024-01-03,Netflix,15.99,Entertainment
2024-01-04,Starbucks,4.50,Eating Out
2024-01-05,Verizon Wireless,60.00,Utilities
2024-01-06,Target,85.00,Shopping
2024-01-07,Whole Foods,70.30,Groceries
2024-01-08,AMC Theatres,25.00,Entertainment
2024-01-09,Spotify,9.99,Entertainment
2024-01-10,Chick-fil-A,18.40,Eating Out
2024-01-11,ExxonMobil,50.00,Transport

items = {
    'Groceries': 105.50,
    'Transport': 95.00,
    'Entertainment': 50.98,
    'Eating Out': 22.90,
    'Utilities': 60.00,
    'Shopping': 85.00
}

total = 0

for category in items:
    total += items[category]

print("Total spending by category:")
for category in items:
    print(category + ": $" + str(items[category]))

print("Total: $" + str(total))

> Finished chain.


'\nitems = {\n    \'Groceries\': 105.50,\n    \'Transport\': 95.00,\n    \'Entertainment\': 50.98,\n    \'Eating Out\': 22.90,\n    \'Utilities\': 60.00,\n    \'Shopping\': 85.00\n}\n\ntotal = 0\n\nfor category in items:\n    total += items[category]\n\nprint("Total spending by category:")\nfor category in items:\n    print(category + ": $" + str(items[category]))\n\nprint("Total: $" + str(total))'